In [1]:
!git clone https://github.com/shiukaheng/minGS.git


fatal: destination path 'minGS' already exists and is not an empty directory.


In [4]:
print("hello world")

hello world


In [2]:
import sys
sys.path.append('/content/minGS')



In [3]:
# make sure we have a compiler helper
!pip install ninja  # speeds up CUDA builds

# --- diff-gaussian-rasterization
!pip install -e ./minGS/submodules/diff-gaussian-rasterization

# --- simple-knn
!pip install -e ./minGS/submodules/simple-knn

Obtaining file:///content/minGS/submodules/diff-gaussian-rasterization
  Preparing metadata (setup.py) ... done
  Attempting uninstall: diff_gaussian_rasterization
    Found existing installation: diff_gaussian_rasterization 0.0.0
    Uninstalling diff_gaussian_rasterization-0.0.0:
      Successfully uninstalled diff_gaussian_rasterization-0.0.0
  Running setup.py develop for diff_gaussian_rasterization
Obtaining file:///content/minGS/submodules/simple-knn
  Preparing metadata (setup.py) ... done
  Attempting uninstall: simple_knn
    Found existing installation: simple_knn 0.0.0
    Uninstalling simple_knn-0.0.0:
      Successfully uninstalled simple_knn-0.0.0
  Running setup.py develop for simple_knn


In [4]:
!pip install plyfile lpips viser requests imageio matplotlib --q


## Baseline script to train **minGS** on a small COLMAP dataset.
- installs the minimal 3‑D Gaussian Splatting implementation
- downloads a tiny demo dataset (bonsai) if `DATA_DIR` is empty
- runs a very short training loop (2 k iters)
- dumps preview renderings every 100 steps
- saves the final model to `aquatic_baseline_gs.pth`

You can use the same scaffolding later and simply swap the `GaussianModel.forward` call with your deflection‑aware version.


# data

In [5]:
# autoreload
%reload_ext autoreload
%autoreload 2

In [11]:
# aquatic_gs_baseline.py – baseline Gaussian Splatting demo (using Tanks&Temples 'Barn' COLMAP dataset)
# Automatically install missing dependencies and run Gaussian Splatting demo for 'Barn'.

# ------- Dependencies installation (run once at start) ------------------------
import importlib
import subprocess
import sys

# List of PyPI packages and minGS repo to ensure installed
REQUIRED_PKGS = [
    "git+https://github.com/shiukaheng/minGS.git",  # Gaussian Splatting core
    "plyfile",                                       # COLMAP PLY support
    "lpips",                                         # perceptual loss
    "viser",                                         # visualization helper
    "requests",                                      # HTTP
    "imageio",                                       # image I/O
    "matplotlib"                                     # plotting
]

for pkg in REQUIRED_PKGS:
    name = pkg.split("#")[0].strip()
    try:
        if name.startswith("git+"):
            importlib.import_module("gs")
        else:
            importlib.import_module(name)
    except ImportError:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

# ------- Imports --------------------------------------------------------------
import os,tempfile, zipfile, requests, imageio, torch
from pathlib import Path
from plyfile import PlyElement

from gs.core.GaussianModel import GaussianModel
from gs.io.colmap import load
from gs.helpers.loss import l1_loss

# ------- Prepare dataset ------------------------------------------------------
# We'll download the authors' Tanks&Temples+DeepBlending COLMAP archive and extract only the 'tandt' scene
ROOT = Path("./data")
DATA_DIR = Path("./data/tandt/train")
if not DATA_DIR.exists():
    print("Dataset not found – downloading precomputed COLMAP for Tanks&Temples (650 MB)…")
    url = "https://repo-sam.inria.fr/fungraph/3d-gaussian-splatting/datasets/input/tandt_db.zip"
    tmpfile = tempfile.NamedTemporaryFile(suffix=".zip", delete=False).name

    resp = requests.get(url, stream=True)
    resp.raise_for_status()
    with open(tmpfile, "wb") as f:
        for chunk in resp.iter_content(chunk_size=8192):
            f.write(chunk)

    with zipfile.ZipFile(tmpfile, 'r') as z:
        members = [m for m in z.namelist()]
        z.extractall(path=str(ROOT), members=members)
    os.remove(tmpfile)

# ------- Monkey-patch missing CUDA distance function --------------------------
# import gs.core.GaussianModel as _gm
# import torch as _torch
# if not hasattr(_gm, 'distCUDA2'):
#     def distCUDA2(a, b):
#         return _torch.cdist(a, b)
#     _gm.distCUDA2 = distCUDA2

# ------- Load COLMAP data -----------------------------------------------------
# Expects: BARN_DIR/sparse/0/{cameras.bin, images.bin, points3D.bin}, BARN_DIR/images/*
cameras, pointcloud = load(str(DATA_DIR))
print(f"Loaded {len(cameras)} cameras and {len(pointcloud.points)} sparse points from Barn")


# ------- Build Gaussian Splatting model ---------------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
model = GaussianModel.from_point_cloud(pointcloud).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, eps=1e-15)

# ------- Quick training loop --------------------------------------------------
NUM_ITERS = 20000  # keep small for baseline; bump for quality
LOG_EVERY = 1000

for it in range(NUM_ITERS):
    cam = cameras[it % len(cameras)].to(device)
    rgb_pred = model(cam)
    loss = l1_loss(rgb_pred, cam.image.to(device))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad(set_to_none=True)

    if (it + 1) % LOG_EVERY == 0 or it == 0:
        print(f"iter {it+1:4d}/{NUM_ITERS} | L1 = {loss.item():.4f}")
        vis = (rgb_pred.clamp(0, 1).permute(1, 2, 0).detach().cpu().numpy() * 255).astype('uint8')
        torch.save(model.state_dict(), "aquatic_baseline_gs.pth")


        imageio.imwrite(f"preview_{it+1:04d}.png", vis)

# ------- Save model -----------------------------------------------------------
torch.save(model.state_dict(), "aquatic_baseline_gs.pth")
print("Training finished – previews and checkpoint written.")


Loaded 301 cameras and 182686 sparse points from Barn
iter    1/20000 | L1 = 0.2476
iter 1000/20000 | L1 = 0.1143
iter 2000/20000 | L1 = 0.0780
iter 3000/20000 | L1 = 0.1275
iter 4000/20000 | L1 = 0.0925
iter 5000/20000 | L1 = 0.0820
iter 6000/20000 | L1 = 0.0568
iter 7000/20000 | L1 = 0.0795
iter 8000/20000 | L1 = 0.0779
iter 9000/20000 | L1 = 0.0418
iter 10000/20000 | L1 = 0.0763
iter 11000/20000 | L1 = 0.0681
iter 12000/20000 | L1 = 0.0509
iter 13000/20000 | L1 = 0.0722
iter 14000/20000 | L1 = 0.0917
iter 15000/20000 | L1 = 0.0697
iter 16000/20000 | L1 = 0.0584
iter 17000/20000 | L1 = 0.0814
iter 18000/20000 | L1 = 0.1784
iter 19000/20000 | L1 = 0.0431
iter 20000/20000 | L1 = 0.0660
Training finished – previews and checkpoint written.


#nice visuals

In [20]:
import numpy as np
import torch
from gs.core.BaseCamera import BaseCamera

# --- add a drop-in replacement ------------------------------------------------
def _look_at(
        eye: torch.Tensor,          # (3,)  camera position
        center: torch.Tensor,       # (3,)  look-at point
        up: torch.Tensor,           # (3,)  world-up
        width: int, height: int,    # image size (px)
        fx: float                   # focal length in px (assume square pixels)
):
    eye_np, center_np, up_np = map(lambda v: v.detach().cpu().numpy(), (eye, center, up))

    # Build rotation (camera → world) with the usual OpenGL look-at convention
    f = center_np - eye_np
    f /= np.linalg.norm(f)
    s = np.cross(f, up_np)
    s /= np.linalg.norm(s)
    u = np.cross(s, f)

    # Camera‐to-world (R^T | eye); we need world-to-camera, so take the transpose.
    R_c2w = np.stack([s, u, -f], axis=0)          # shape (3, 3)
    R = R_c2w.T                                   # world → camera
    t = -R @ eye_np                               # translation in camera frame

    fov_x = 2 * np.arctan(width  / (2 * fx))
    fov_y = 2 * np.arctan(height / (2 * fx))

    return BaseCamera(
        image_height=height,
        image_width=width,
        fov_x=float(fov_x),
        fov_y=float(fov_y),
        R=R.astype(np.float32),
        t=t.astype(np.float32),
    )

# attach it so the rest of the code can stay identical
BaseCamera.look_at = staticmethod(_look_at)
# ------------------------------------------------------------------------------



In [118]:
import numpy as np
import torch
from gs.core.BaseCamera import BaseCamera

def orbit_around_camera(
        ref_cam: BaseCamera,
        radius: float,
        n_views: int = 120,
        axis: str = "yaw",            # "yaw"  (spin left/right)
                                        # "pitch" (tilt up/down)
                                        # "roll"  (spin around optical axis)
        up_hint: torch.Tensor | None = None,   # world-space up  (optional)
        width: int = 800,
        height: int = 800,
        fx: float = 700,
) -> list[BaseCamera]:

    # -- extract the reference-camera transforms -----------------------------
    R_wc = ref_cam.world_view_transform[:3, :3].T      # world←camera
    t_wc = ref_cam.camera_center                       # camera position (world)

    # local basis: x = right, y = up, z = -view
    axis_vectors = {
        "yaw":   torch.tensor([0, 1, 0]).to(device),   # rotate around local Y-up
        "pitch": torch.tensor([1, 0, 0]).to(device),   # rotate around local X-right
        "roll":  torch.tensor([0, 0, 1]).to(device),   # rotate around local Z-view
    }
    a_local = axis_vectors[axis].float()

    # up vector for look_at; fall back to camera’s own +Y if none supplied
    up_world = up_hint if up_hint is not None else R_wc @ torch.tensor([0, 1, 0],dtype=torch.float).to(device)

    cams = []
    for theta in torch.linspace(0.9*np.pi, 1.5*np.pi, n_views):
        # rotation matrix in *camera* space, then send to world space
        cos_t, sin_t = torch.cos(theta), torch.sin(theta)
        if axis == "yaw":
            R_delta = torch.tensor([[ cos_t, 0, sin_t],
                                    [     0, 1,     0],
                                    [-sin_t, 0, cos_t]]).to(device)
        elif axis == "pitch":
            R_delta = torch.tensor([[1,     0,      0],
                                    [0, cos_t, -sin_t],
                                    [0, sin_t,  cos_t]]).to(device)
        else:  # roll
            R_delta = torch.tensor([[cos_t, -sin_t, 0],
                                    [sin_t,  cos_t, 0],
                                    [    0,      0, 1]]).to(device)

        # eye position: start at (radius,0,0) in ref-cam space, rotate, go world
        eye_local  = torch.tensor([radius, 0, 0]).to(device)
        eye_world  = (R_wc @ (R_delta @ eye_local)) + t_wc

        # look-at point = pivot itself (camera centre)
        cams.append(
            BaseCamera.look_at(
                eye_world, t_wc, up_world,
                width=width, height=height, fx=fx
            ).to("cuda")
        )

    return cams


# get the video

In [119]:
# choose any dataset camera you like (e.g. index 0)
ref_cam = cameras[16]

# build an orbit that yaws 360° around that camera
cams = orbit_around_camera(
    ref_cam,
    radius=2.0,      # metres in *camera* space
    n_views=120,
    axis="yaw",       # try "pitch" or "roll" for other motions
    width=800, height=800, fx=700
)

# render exactly as before
frames = []
for cam in cams:
    with torch.no_grad():
        img = model(cam).clamp(0,1).permute(1,2,0).cpu().numpy()
    frames.append((img*255).astype("uint8"))

imageio.mimsave("pivot_turntable.mp4", frames, fps=24)
print("Saved pivot_turntable.mp4")


Saved pivot_turntable.mp4


____________